In [1]:
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt

#Text Color
from termcolor import colored

#Train Test Split
from sklearn.model_selection import train_test_split

#Model Evaluation
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score


#Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Embedding

In [2]:
pd1 = pd.read_csv("train.csv")
pd2 = pd.read_csv("test.csv")

In [3]:
X_train = pd1['Title']+" "+pd1['Description']
X_test = pd2 ['Title']+" "+pd2['Description']

In [10]:
y_train  =   pd1['Class Index'].apply(lambda x: x-1)
y_test =    pd2['Class Index'].apply(lambda x: x-1) 

In [4]:
max_len = X_train.map(lambda x : len(x.split())).max()

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocabulary_size = 10000 # random value
embed_size      = 32    # random value 

tok = Tokenizer(num_words=vocabulary_size)
tok.fit_on_texts(X_train.values)


# Token 
X_train = tok.texts_to_sequences(X_train)
X_test  = tok.texts_to_sequences(X_test)

# Now we need to pad all the sequences based on the max value 

X_train = pad_sequences(X_train,maxlen=max_len)
X_test = pad_sequences(X_test,maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocabulary_size,embed_size,input_length = max_len)) #input layer is embedding layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))              # Bidirectinal LSTM
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPooling1D())                                         # Flattening layer to reduce everything in a vector form
model.add(Dense(256, activation='relu'))                                                  # Dense layer
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.25))                                                # doing regularization in Neural Network
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))                               #  we have 4 labels as outputmodel.summary()

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 177, 32)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 177, 256)          164864    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 177, 128)          164352    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3

In [8]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy']             
             )

In [11]:
model.fit(X_train,y_train,batch_size=256,validation_data=(X_test,y_test),epochs=20)


Epoch 1/20
469/469 [==============================] - 701s 1s/step - loss: 0.5424 - accuracy: 0.7710 - val_loss: 0.2775 - val_accuracy: 0.9097
Epoch 2/20
262/469 [===============>..............] - ETA: 3:41 - loss: 0.2519 - accuracy: 0.9222

KeyboardInterrupt: 